# Imports

In [1]:
import os
import tarfile
import urllib
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin

# Data Fetching

In [2]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("..", "data", "datasets", "housing", "raw")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

In [3]:
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()


def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [4]:
housing = load_housing_data(housing_path=HOUSING_PATH)

# Train Test Splitting

In [5]:
housing["income_cat"] = pd.cut(
    housing["median_income"],
    bins=[0.0, 1.5, 3.0, 4.5, 6.0, np.inf],
    labels=[1, 2, 3, 4, 5],
)

In [6]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

# Custom Pipeline

In [7]:
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6


class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):  # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room

    def fit(self, X, y=None):
        return self  # nothing else to do

    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[
                X,
                rooms_per_household,
                population_per_household,
                bedrooms_per_room,
            ]

        else:
            return np.c_[X, rooms_per_household, population_per_household]


attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=True)
housing_extra_attribs = attr_adder.transform(housing.values)

In [8]:
housing_num = housing.drop("ocean_proximity", axis=1)

num_pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("attribs_adder", CombinedAttributesAdder()),
        # ('std_scaler', StandardScaler()),
    ]
)

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer(
    [
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ]
)

In [9]:
# housing_num_tr = num_pipeline.fit_transform(housing_num)

housing_prepared = pd.DataFrame(
    full_pipeline.fit_transform(housing),
    columns=[
        "longitude",
        "latitude",
        "housing_median_age",
        "total_rooms",
        "total_bedrooms",
        "population",
        "households",
        "median_income",
        "rooms_per_household",
        "population_per_household",
        "bedrooms_per_room",
        "<1H OCEAN",
        "INLAND",
        "ISLAND",
        "NEAR BAY",
        "NEAR OCEAN",
    ],
)
housing_prepared.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_household,population_per_household,bedrooms_per_room,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,5.485836,3.168555,0.205784,0.0,1.0,0.0,0.0,0.0
1,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,6.927083,2.623698,0.160714,0.0,0.0,0.0,0.0,1.0
2,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,5.393333,2.223333,0.191595,0.0,1.0,0.0,0.0,0.0
3,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,3.886128,1.859213,0.276505,0.0,0.0,0.0,0.0,1.0
4,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,6.096552,3.167241,0.182692,1.0,0.0,0.0,0.0,0.0


# 

## 1. 

Question: Try a Support Vector Machine regressor (`sklearn.svm.SVR`), with various hyperparameters such as `kernel="linear"` (with various values for the `C` hyperparameter) or `kernel="rbf"` (with various values for the `C` and `gamma` hyperparameters). Don't worry about what these hyperparameters mean for now. How does the best `SVR` predictor perform?

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

In [16]:
param_grid = [
    {"kernel": ["linear"], "C": [1000.0, 3000.0, 10000.0, 30000.0]},
    {
        "kernel": ["rbf"],
        "C": [100.0, 300.0, 1000.0],
        "gamma": [0.03, 0.1, 0.3, 1.0],
    },
]

svm_reg = SVR()
grid_search = GridSearchCV(
    svm_reg, param_grid, cv=5, scoring="neg_mean_squared_error", verbose=2
)
grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
negative_mse = grid_search.best_score_
rmse = np.sqrt(-negative_mse)
rmse

## 2.
Question: Try replacing GridSearchCV with RandomizedSearchCV.

In [33]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, reciprocal

param_distribs = {
    "kernel": ["linear", "rbf"],
    "C": reciprocal(20, 200000),
    "gamma": expon(scale=1.0),
}

svm_reg = SVR()
rnd_search = RandomizedSearchCV(
    svm_reg,
    param_distributions=param_distribs,
    n_iter=50,
    cv=5,
    scoring="neg_mean_squared_error",
    verbose=2,
    random_state=42,
)
rnd_search.fit(housing_prepared, housing_labels)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


In [ ]:
rnd_search.best_params_

{'max_features': 7, 'n_estimators': 180}

In [32]:
negative_mse = rnd_search.best_score_
rmse = np.sqrt(-negative_mse)
rmse

49116.88125713075

## 3.
Question: Try adding a transformer in the preparation pipeline to select only the most important attributes.

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin


def indices_of_top_k(arr, k):
    return np.sort(np.argpartition(np.array(arr), -k)[-k:])


class TopFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_importances, k):
        self.feature_importances = feature_importances
        self.k = k

    def fit(self, X, y=None):
        self.feature_indices_ = indices_of_top_k(
            self.feature_importances, self.k
        )
        return self

    def transform(self, X):
        return X[:, self.feature_indices_]

In [14]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
    "n_estimators": randint(low=1, high=200),
    "max_features": randint(low=1, high=8),
}

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(
    forest_reg,
    param_distributions=param_distribs,
    n_iter=10,
    cv=5,
    scoring="neg_mean_squared_error",
    random_state=42,
)
rnd_search.fit(housing_prepared, housing_labels)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   param_distributions={'max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fae2f6ce5d0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fae2f6d3010>},
                   random_state=42, scoring='neg_mean_squared_error')

In [15]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([6.96542523e-02, 6.04213840e-02, 4.21882202e-02, 1.52450557e-02,
       1.55545295e-02, 1.58491147e-02, 1.49346552e-02, 3.79009225e-01,
       5.47789150e-02, 1.07031322e-01, 4.82031213e-02, 6.79266007e-03,
       1.65706303e-01, 7.83480660e-05, 1.52473276e-03, 3.02816106e-03])

In [20]:
k = 5
attributes = housing_prepared.columns
top_k_feature_indices = indices_of_top_k(feature_importances, k)
top_k_feature_indices

array([ 0,  1,  7,  9, 12])

In [21]:
np.array(attributes)[top_k_feature_indices]

array(['longitude', 'latitude', 'median_income',
       'population_per_household', 'INLAND'], dtype=object)

In [22]:
sorted(zip(feature_importances, attributes), reverse=True)[:k]

[(0.3790092248170967, 'median_income'),
 (0.16570630316895876, 'INLAND'),
 (0.10703132208204354, 'population_per_household'),
 (0.06965425227942929, 'longitude'),
 (0.0604213840080722, 'latitude')]

In [23]:
preparation_and_feature_selection_pipeline = Pipeline(
    [
        ("preparation", full_pipeline),
        ("feature_selection", TopFeatureSelector(feature_importances, k)),
    ]
)

In [24]:
housing_prepared_top_k_features = (
    preparation_and_feature_selection_pipeline.fit_transform(housing)
)

In [25]:
housing_prepared_top_k_features[0:3]

array([[-121.46      ,   38.52      ,    2.1736    ,    3.16855524,
           1.        ],
       [-117.23      ,   33.09      ,    6.3373    ,    2.62369792,
           0.        ],
       [-119.04      ,   35.37      ,    2.875     ,    2.22333333,
           1.        ]])

## 4.
Question: Try creating a single pipeline that does the full data preparation plus the final prediction.

In [ ]:
prepare_select_and_predict_pipeline = Pipeline(
    [
        ("preparation", full_pipeline),
        ("feature_selection", TopFeatureSelector(feature_importances, k)),
        ("svm_reg", SVR(**rnd_search.best_params_)),
    ]
)

In [ ]:
prepare_select_and_predict_pipeline.fit(housing, housing_labels)

In [ ]:
some_data = housing.iloc[:4]
some_labels = housing_labels.iloc[:4]

print("Predictions:\t", prepare_select_and_predict_pipeline.predict(some_data))
print("Labels:\t\t", list(some_labels))

## 5.
Question: Automatically explore some preparation options using GridSearchCV.

In [ ]:
full_pipeline.named_transformers_["cat"].handle_unknown = "ignore"

param_grid = [
    {
        "preparation__num__imputer__strategy": [
            "mean",
            "median",
            "most_frequent",
        ],
        "feature_selection__k": list(range(1, len(feature_importances) + 1)),
    }
]

grid_search_prep = GridSearchCV(
    prepare_select_and_predict_pipeline,
    param_grid,
    cv=5,
    scoring="neg_mean_squared_error",
    verbose=2,
)
grid_search_prep.fit(housing, housing_labels)

In [ ]:
grid_search_prep.best_params_